# comprendre la donnée

In [1]:
import pandas as pd

In [2]:
df_meta = pd.read_csv('./birdclef-2024/train_metadata.csv')
df_birbs = pd.read_csv('./birdclef-2024/eBird_Taxonomy_v2021.csv')

In [3]:
df_meta.groupby('primary_label')

In [4]:
# show unique primary labels
# print(df_meta['primary_label'].unique())

# for each primary label, show the number of samples
print(df_meta['primary_label'].value_counts())



primary_label
zitcis1    500
lirplo     500
litgre1    500
comgre     500
comkin1    500
          ... 
blaeag1      6
wynlau1      6
niwpig1      5
asiope1      5
integr       5
Name: count, Length: 182, dtype: int64


In [5]:
# secondary labels are in the form of a list of strings, so we need to split them into separate columns to count them
df_meta['secondary_labels'] = df_meta['secondary_labels'].str.split('|')
df_meta = df_meta.explode('secondary_labels')

In [6]:
df_meta.columns

Index(['primary_label', 'secondary_labels', 'type', 'latitude', 'longitude',
       'scientific_name', 'common_name', 'author', 'license', 'rating', 'url',
       'filename'],
      dtype='object')

# CNN test 1


In [7]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, concatenate
from tensorflow.keras.optimizers import Adam

In [8]:

def load_audio(file_path, sr=22050):
    audio, sr = librosa.load(file_path, sr=sr)
    return audio, sr

def get_spectrogram(audio, sr=22050, n_mels=128, fmax=8000):
    spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=n_mels, fmax=fmax)
    return spectrogram

def spectral_gate(spec, noise_mean, threshold):
    spec_denoised = spec - noise_mean[:, np.newaxis] * threshold
    spec_denoised[spec_denoised < 0] = 0
    return spec_denoised

def apply_augmentation(spec):
    noise = np.random.randn(*spec.shape) * 0.01
    spec_noise_aug = spec + noise
    spec_distort1 = spec * (1 + 0.1 * np.sin(2 * np.pi * np.arange(spec.shape[1]) / 50))
    spec_distort2 = spec * (1 + 0.1 * np.cos(2 * np.pi * np.arange(spec.shape[1]) / 50))
    return spec_noise_aug, spec_distort1, spec_distort2

def generate_spectrogram(row, audio_len=22050*5, n_mels=128, fmax=8000):
    audio, sr = load_audio(row['file_path'])
    audio = audio[:audio_len]
    spec = get_spectrogram(audio, sr=sr, n_mels=n_mels, fmax=fmax)
    spec_db = librosa.power_to_db(spec, ref=np.max)
    return spec_db

In [9]:
class BirdSpeciesClassifier:
    def __init__(self, metadata_path, audio_dir, num_classes, input_shape=(128, 128, 1), batch_size=32, epochs=10):
        self.metadata_path = metadata_path
        self.audio_dir = audio_dir
        self.num_classes = num_classes
        self.input_shape = input_shape
        self.batch_size = batch_size
        self.epochs = epochs
        self.meta_columns = ['latitude', 'longitude']  # Define meta_columns here
        self.model = self.build_model()
        
    def build_model(self):
        # Image input branch
        img_input = Input(shape=self.input_shape, name='img_input')
        x = Conv2D(32, (3, 3), activation='relu')(img_input)
        x = MaxPooling2D((2, 2))(x)
        x = Dropout(0.25)(x)
        x = Conv2D(64, (3, 3), activation='relu')(x)
        x = MaxPooling2D((2, 2))(x)
        x = Dropout(0.25)(x)
        x = Conv2D(128, (3, 3), activation='relu')(x)
        x = MaxPooling2D((2, 2))(x)
        x = Dropout(0.25)(x)
        x = Flatten()(x)
        
        # Metadata input branch
        meta_input = Input(shape=(len(self.meta_columns),), name='meta_input')
        y = Dense(128, activation='relu')(meta_input)
        y = Dropout(0.5)(y)
        
        # Concatenate the outputs
        combined = concatenate([x, y])
        z = Dense(128, activation='relu')(combined)
        z = Dropout(0.5)(z)
        z = Dense(self.num_classes, activation='softmax')(z)
        
        model = Model(inputs=[img_input, meta_input], outputs=z)
        model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
        return model
    
    def preprocess_metadata(self, df):
        meta_data = df[self.meta_columns].values
        labels = df['primary_label'].astype('category').cat.codes
        labels = to_categorical(labels, num_classes=self.num_classes)
        return meta_data, labels
    
    def spectrogram_generator(self, df):
        num_samples = len(df)
        while True:
            df = df.sample(frac=1).reset_index(drop=True)
            for offset in range(0, num_samples, self.batch_size):
                batch_df = df.iloc[offset:offset+self.batch_size]
                batch_spectrograms = []
                batch_meta_data = []
                batch_labels = []
                for _, row in batch_df.iterrows():
                    spec = generate_spectrogram(row)
                    batch_spectrograms.append(spec)
                    batch_meta_data.append(row[self.meta_columns])
                    batch_labels.append(row['primary_label'])
                
                X_img = np.array(batch_spectrograms)[..., np.newaxis]
                X_meta = np.array(batch_meta_data)
                y = to_categorical(np.array(batch_labels).astype('category').cat.codes, num_classes=self.num_classes)
                
                yield {'img_input': X_img, 'meta_input': X_meta}, y
    
    def train(self):
        df = pd.read_csv(self.metadata_path)
        train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
        train_meta, train_labels = self.preprocess_metadata(train_df)
        val_meta, val_labels = self.preprocess_metadata(val_df)
        
        steps_per_epoch = len(train_df) // self.batch_size
        validation_steps = len(val_df) // self.batch_size
        
        self.model.fit(
            self.spectrogram_generator(train_df),
            steps_per_epoch=steps_per_epoch,
            validation_data=self.spectrogram_generator(val_df),
            validation_steps=validation_steps,
            epochs=self.epochs
        )


In [10]:
# Assuming 'num_classes' is the number of unique bird species
classifier = BirdSpeciesClassifier(
    metadata_path='./birdclef-2024/train_metadata.csv',
    audio_dir='./birdclef-2024/train_audio/',
    num_classes=df_meta['primary_label'].nunique(),
    input_shape=(128, 128, 1),
    batch_size=32,
    epochs=10
)

classifier.train()


NameError: name 'CFG' is not defined